In [1]:
from typing import Iterator, Any

from nltk import Tree
from nltk.grammar import FeatureGrammar
from nltk.parse import FeatureEarleyChartParser
from nltk.parse.generate import generate

from utils.abstract_factory import factory_function

# Exemple d'une grammaire prenant en compte les ambiguité morphologique ET syntaxique
Avec un vocabulaire comme le/la/les/chat/chats/cuisine/cuisines/dort/dorment/dans, on peut vite exploser la combinatoire.

In [2]:
def generate_grammar_productions(g, depth):
    memo = set()
    for x in generate(g, depth=depth):
        if " ".join(x) in memo:
            continue
        else:
            memo.add(" ".join(x))
        trees = [*parser.parse(x)]
        if trees:
            print(" ".join(x))
            print()
            for tree in trees:
                print(" ".join(tree.label()["KA", "Traduction"]))
            print()
            print()

In [3]:
def pprint(id_print: str, trees: Iterator[Tree]) -> Any:
    """ factory pour afficher de différentes façons les arbres d'un parsing """
    return factory_function(concrete_product=f"print_{id_print}",
                            package=__name__,
                            trees=trees)


def print_possible_trees(trees: Iterator[Tree]) -> None:
    """
    pour chaque arbre, on dessine l'arbre associé (style HPSG)
    Attention, il faut quitter l'arbre, pour que le suivant apparaisse.
    Il faut donc quitter le dernier arbre à afficher pour quitter cette fonction (ou tuer le process qui exécute cette fonction)
    """
    for t in trees:
        t.draw()


def print_traduction(trees: Iterator[Tree]) -> None:
    """ pour chaque arbre, on affiche les traductions associées """
    for t in trees:
        print(t.label()["KA", "Traduction"])


def print_syntagmes(trees: Iterator[Tree]) -> None:
    """ pour chaque arbre, on affiche les syntagmes sous formes parenthésée """
    for t in trees:
        print(t.label()["KA", "Syntagmes"])

In [4]:
gram = """% start S

S[KA=[Traduction=(?np+?vp),Syntagmes=(?np,?vp)],FR=[]] -> NP[KA=[Genre=?tgenre,Nombre=?tnombre,Traduction=?np],FR=[Nombre=?nombre,Personne=?personne]] VP[KA=[Genre=?tgenre,Nombre=?tnombre,Traduction=?vp],FR=[Nombre=?nombre,Personne=?personne]]

S[KA=[Traduction=(?np+?vp+?pp),Syntagmes=(?np,?vp,?pp)],FR=[]] -> NP[KA=[Genre=?tgenre,Nombre=?tnombre,Traduction=?np],FR=[Nombre=?nombre,Personne=?personne]] VP[KA=[Genre=?tgenre,Nombre=?tnombre,Traduction=?vp],FR=[Nombre=?nombre,Personne=?personne]] PP[KA=[Traduction=?pp]]

S[KA=[Traduction=(?np+?vp+?pp),Syntagmes=(?np,?vp,?pp)],FR=[]] -> NP[KA=[Genre=?tgenre,Nombre=?tnombre,Traduction=?np],FR=[Nombre=?nombre,Personne=?personne]] PP[KA=[Traduction=?pp]] VP[KA=[Genre=?tgenre,Nombre=?tnombre,Traduction=?vp],FR=[Nombre=?nombre,Personne=?personne]]

VP[KA=[Genre=?tgenre,Nombre=?tnombre,Traduction=?v,Syntagmes=?v],FR=[Nombre=?nombre,Personne=?personne]] -> V[KA=[Genre=?tgenre,Nombre=?tnombre,Traduction=?v,Syntagmes=?v],FR=[Nombre=?nombre,Personne=?personne]]

PP[KA=[Traduction=(?prep+?np),Syntagmes=(?prep,((?np)))]] -> PREP[KA=[Traduction=?prep]] NP[KA=[Traduction=?np]]

NP[KA=[Genre=?tgenre,Nombre=?tnombre,Traduction=(?det+?n),Syntagmes=(?det,?n)],FR=[Genre=?genre,Nombre=?nombre]] -> DET[KA=[Genre=?tgenre,Nombre=?tnombre,Traduction=?det],FR=[Genre=?genre,Nombre=?nombre]] N[KA=[Genre=?tgenre,Nombre=?tnombre,Traduction=?n],FR=[Genre=?genre,Nombre=?nombre]]

NP[KA=[Genre=?tgenre,Nombre=?tnombre,Traduction=(?det+?n+?pp),Syntagmes=(?det,?n,?pp)],FR=[Genre=?genre,Nombre=?nombre]] -> DET[KA=[Genre=?tgenre,Nombre=?tnombre,Traduction=?det],FR=[Genre=?genre,Nombre=?nombre]] N[KA=[Genre=?tgenre,Nombre=?tnombre,Traduction=?n],FR=[Genre=?genre,Nombre=?nombre]] PP[KA=[Traduction=?pp]]


PREP[KA=[Traduction='tok',Syntagmes='tok']] -> "dans"

DET[KA=[Genre=1,Nombre=Sg,Traduction='bag',Syntagmes='bag'],FR=[Nombre=Sg,Genre=M]] -> 'le'
DET[KA=[Genre=1,Nombre=Pl,Traduction='bat',Syntagmes='bat'],FR=[Nombre=Sg,Genre=M]] -> 'le'
DET[KA=[Genre=2,Nombre=Sg,Traduction='bug',Syntagmes='bug'],FR=[Nombre=Sg,Genre=M]] -> 'le'
DET[KA=[Genre=2,Nombre=Pl,Traduction='but',Syntagmes='but'],FR=[Nombre=Sg,Genre=M]] -> 'le'
DET[KA=[Genre=1,Nombre=Sg,Traduction='bag',Syntagmes='bag'],FR=[Nombre=Pl,Genre=M]] -> 'les'
DET[KA=[Genre=1,Nombre=Pl,Traduction='bat',Syntagmes='bat'],FR=[Nombre=Pl,Genre=M]] -> 'les'
DET[KA=[Genre=2,Nombre=Sg,Traduction='bug',Syntagmes='bug'],FR=[Nombre=Pl,Genre=M]] -> 'les'
DET[KA=[Genre=2,Nombre=Pl,Traduction='but',Syntagmes='but'],FR=[Nombre=Pl,Genre=M]] -> 'les'

DET[KA=[Genre=1,Nombre=Sg,Traduction='bag',Syntagmes='bag'],FR=[Nombre=Sg,Genre=F]] -> 'la'
DET[KA=[Genre=1,Nombre=Pl,Traduction='bat',Syntagmes='bat'],FR=[Nombre=Sg,Genre=F]] -> 'la'
DET[KA=[Genre=2,Nombre=Sg,Traduction='bug',Syntagmes='bug'],FR=[Nombre=Sg,Genre=F]] -> 'la'
DET[KA=[Genre=2,Nombre=Pl,Traduction='but',Syntagmes='but'],FR=[Nombre=Sg,Genre=F]] -> 'la'
DET[KA=[Genre=1,Nombre=Sg,Traduction='bag',Syntagmes='bag'],FR=[Nombre=Pl,Genre=F]] -> 'les'
DET[KA=[Genre=1,Nombre=Pl,Traduction='bat',Syntagmes='bat'],FR=[Nombre=Pl,Genre=F]] -> 'les'
DET[KA=[Genre=2,Nombre=Sg,Traduction='bug',Syntagmes='bug'],FR=[Nombre=Pl,Genre=F]] -> 'les'
DET[KA=[Genre=2,Nombre=Pl,Traduction='but',Syntagmes='but'],FR=[Nombre=Pl,Genre=F]] -> 'les'

N[KA=[Genre=1,Nombre=Sg,Traduction='gato',Syntagmes='gato'],FR=[Nombre=Sg,Genre=M]] -> 'chat'
N[KA=[Genre=1,Nombre=Sg,Traduction='gato',Syntagmes='gato'],FR=[Nombre=Pl,Genre=M]] -> 'chats'
N[KA=[Genre=1,Nombre=Pl,Traduction='gatos',Syntagmes='gatos'],FR=[Nombre=Sg,Genre=M]] -> 'chat'
N[KA=[Genre=1,Nombre=Pl,Traduction='gatos',Syntagmes='gatos'],FR=[Nombre=Pl,Genre=M]] -> 'chats'

N[KA=[Genre=2,Nombre=Sg,Traduction='minet',Syntagmes='minet'],FR=[Nombre=Sg,Genre=F]] -> 'cuisine'
N[KA=[Genre=2,Nombre=Sg,Traduction='minet',Syntagmes='minet'],FR=[Nombre=Pl,Genre=F]] -> 'cuisines'
N[KA=[Genre=2,Nombre=Pl,Traduction='minets',Syntagmes='minets'],FR=[Nombre=Sg,Genre=F]] -> 'cuisine'
N[KA=[Genre=2,Nombre=Pl,Traduction='minets',Syntagmes='minets'],FR=[Nombre=Pl,Genre=F]] -> 'cuisines'

V[KA=[Genre=1,Nombre=Sg,Traduction='umamati',Syntagmes='umamati'],FR=[Nombre=Sg,Personne=3]] -> 'dort'
V[KA=[Genre=1,Nombre=Sg,Traduction='umamati',Syntagmes='umamati'],FR=[Nombre=Pl,Personne=3]] -> 'dorment'
V[KA=[Genre=1,Nombre=Pl,Traduction='umamatu',Syntagmes='umamatu'],FR=[Nombre=Sg,Personne=3]] -> 'dort'
V[KA=[Genre=1,Nombre=Pl,Traduction='umamatu',Syntagmes='umamatu'],FR=[Nombre=Pl,Personne=3]] -> 'dorment'
V[KA=[Genre=2,Nombre=Sg,Traduction='mamati',Syntagmes='mamati'],FR=[Nombre=Sg,Personne=3]] -> 'dort'
V[KA=[Genre=2,Nombre=Sg,Traduction='mamati',Syntagmes='mamati'],FR=[Nombre=Pl,Personne=3]] -> 'dorment'
V[KA=[Genre=2,Nombre=Pl,Traduction='mamatu',Syntagmes='mamatu'],FR=[Nombre=Sg,Personne=3]] -> 'dort'
V[KA=[Genre=2,Nombre=Pl,Traduction='mamatu',Syntagmes='mamatu'],FR=[Nombre=Pl,Personne=3]] -> 'dorment'"""

In [5]:
gg = FeatureGrammar.fromstring(gram)
parser = FeatureEarleyChartParser(gg)

## Les deux exemples suivants sont ambigus morphologiquement
### La phrase suivante est ambiguë syntaxiquement

In [9]:
trees = [*parser.parse("les chats dans la cuisine dorment".split(" "))]
pprint(id_print="syntagmes" ,trees=trees)
print()
pprint(id_print="traduction" ,trees=trees)

((bat, gatos), umamatu, (tok, bug, minet))
((bat, gatos, tok, bug, minet), umamatu)
((bat, gatos), umamatu, (tok, but, minets))
((bat, gatos, tok, but, minets), umamatu)
((bag, gato), umamati, (tok, bug, minet))
((bag, gato, tok, bug, minet), umamati)
((bag, gato), umamati, (tok, but, minets))
((bag, gato, tok, but, minets), umamati)

(bat, gatos, umamatu, tok, bug, minet)
(bat, gatos, tok, bug, minet, umamatu)
(bat, gatos, umamatu, tok, but, minets)
(bat, gatos, tok, but, minets, umamatu)
(bag, gato, umamati, tok, bug, minet)
(bag, gato, tok, bug, minet, umamati)
(bag, gato, umamati, tok, but, minets)
(bag, gato, tok, but, minets, umamati)


### La phrase suivante n'est pas ambiguë syntaxiquement

In [7]:
trees = [*parser.parse("les chats dorment dans la cuisine".strip().split(" "))]
pprint(id_print="syntagmes" ,trees=trees)
print()
pprint(id_print="traduction" ,trees=trees)

((bat, gatos), umamatu, (tok, bug, minet))
((bag, gato), umamati, (tok, bug, minet))
((bat, gatos), umamatu, (tok, but, minets))
((bag, gato), umamati, (tok, but, minets))

(bat, gatos, umamatu, tok, bug, minet)
(bag, gato, umamati, tok, bug, minet)
(bat, gatos, umamatu, tok, but, minets)
(bag, gato, umamati, tok, but, minets)


In [8]:
generate_grammar_productions(gg, 5)


le chat dort

bat gatos umamatu
bag gato umamati


les chats dorment

bat gatos umamatu
bag gato umamati


les cuisines dorment

but minets mamatu
bug minet mamati


la cuisine dort

but minets mamatu
bug minet mamati


le chat dort dans le chat

bat gatos umamatu tok bag gato
bag gato umamati tok bag gato
bat gatos umamatu tok bat gatos
bag gato umamati tok bat gatos


le chat dort dans les chats

bat gatos umamatu tok bag gato
bag gato umamati tok bag gato
bat gatos umamatu tok bat gatos
bag gato umamati tok bat gatos


le chat dort dans les cuisines

bat gatos umamatu tok bug minet
bag gato umamati tok bug minet
bat gatos umamatu tok but minets
bag gato umamati tok but minets


le chat dort dans la cuisine

bat gatos umamatu tok bug minet
bag gato umamati tok bug minet
bat gatos umamatu tok but minets
bag gato umamati tok but minets


les chats dorment dans le chat

bat gatos umamatu tok bag gato
bag gato umamati tok bag gato
bat gatos umamatu tok bat gatos
bag gato umamati tok bat 